In [7]:
import torch
from active_rl.networks import ENS_DQN
from active_rl.trainer import train_AMN
from active_rl.utils import make_raw_env, env_shape, LabeledReplayMemory, AMN_optimization_ensemble, ens_BALD

In [8]:
BATCH_SIZE = 128
LR = 0.0000625
GAMMA = 0.99
EPS = 0.05
NUM_STEPS = 10000000
NOT_LABELLED_CAPACITY = 10000
LABELLED_CAPACITY = 400000
INITIAL_STEPS=NOT_LABELLED_CAPACITY
PERCENTAGE = 0.05
TRAIN_ITERS = int(PERCENTAGE * NOT_LABELLED_CAPACITY / 10)

ENV_NAME = 'Breakout'
NAME = 'AMN_Ensemble_5_perc_labelled'

In [9]:
env_raw = make_raw_env(ENV_NAME)
channel, height, width, n_actions = env_shape(env_raw)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
AMN_net = ENS_DQN(n_actions).to(device)
expert_net = torch.load("models/dqn_expert_breakout_model").to(device)
expert_net.eval()
optimizer = optim.Adam(AMN_net.parameters(), lr=LR, eps=1.5e-4)
train_func = AMN_optimization_ensemble
memory = LabeledReplayMemory(NOT_LABELLED_CAPACITY, LABELLED_CAPACITY, [5 * channel, height, width], 
                             n_actions, ens_BALD, AMN_net, device=device)
action_selector = ActionSelector(AMN_net, EPS, EPS, 1, n_actions, device)
writer = SummaryWriter(f'runs/{NAME}')

ModuleNotFoundError: No module named 'networks'

In [ ]:
train_AMN(AMN_net, expert_net, optimizer, memory, action_selector, TRAIN_ITERS, train_func, env_raw, NUM_STEPS,
         INITIAL_STEPS, PERCENTAGE, NOT_LABELLED_CAPACITY, batch_size=BATCH_SIZE, writer=writer, device=device)